<a href="https://colab.research.google.com/github/aleanaliaaustral-web/labo2025v/blob/main/Arbol%20gridsearch%20consolidado_sin%20semilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ============================================
# Árbol de Regresión: Predecir ganancia_test
# Dataset Consolidado con 10 grupos
============================================

In [1]:
# IMPORTANTE: Primero ejecuta esto en una celda de Python en Colab:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Limpio la memoria
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

# Cargo las librerías necesarias
require("data.table")
require("rpart")
if (!require("rpart.plot")) install.packages("rpart.plot")
require("rpart.plot")

# Crear directorios si no existen
dir.create("/content/buckets", showWarnings=FALSE, recursive=TRUE)
dir.create("/content/buckets/b1", showWarnings=FALSE, recursive=TRUE)
dir.create("/content/buckets/b1/exp", showWarnings=FALSE, recursive=TRUE)

# Carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "KA2004_ArbolGridSearch"
dir.create(experimento, showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento))

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,714662,38.2,1454482,77.7,1454482,77.7
Vcells,1332001,10.2,8388608,64.0,5767534,44.1


In [4]:
# ============================================
# LECTURA DE DATOS
# ============================================

cat("============================================\n")
cat("ÁRBOL DE REGRESIÓN SOBRE GRID SEARCH\n")
cat("============================================\n\n")

# Buscar el archivo en diferentes ubicaciones posibles
rutas_posibles <- c(
  "/content/buckets/b1/datasets/gridsearch_grupodiez.csv",
  "/content/datasets/gridsearch_grupodiez.csv",
  "/content/.drive/My Drive/labo1/datasets/gridsearch_grupodiez.csv",
  "/content/.drive/MyDrive/labo1/datasets/gridsearch_grupodiez.csv"
)

ruta_archivo <- NULL
for (ruta in rutas_posibles) {
  if (file.exists(ruta)) {
    ruta_archivo <- ruta
    cat("Archivo encontrado en:", ruta, "\n\n")
    break
  }
}

if (is.null(ruta_archivo)) {
  stop("ERROR: No se encontró el archivo gridsearch_grupodiez.csv en ninguna ubicación conocida")
}

# Leo el dataset de grid search
dataset <- fread(ruta_archivo)

cat("Dataset leído correctamente!\n")
cat("Dimensiones:", nrow(dataset), "filas x", ncol(dataset), "columnas\n")
cat("Columnas:", paste(names(dataset), collapse=", "), "\n\n")

# Resumen estadístico
cat("Resumen de ganancia_mean:\n")
cat("  Mínimo:  ", format(min(dataset$ganancia_mean), big.mark=","), "\n")
cat("  Máximo:  ", format(max(dataset$ganancia_mean), big.mark=","), "\n")
cat("  Media:   ", format(mean(dataset$ganancia_mean), big.mark=","), "\n")
cat("  Mediana: ", format(median(dataset$ganancia_mean), big.mark=","), "\n\n")

ÁRBOL DE REGRESIÓN SOBRE GRID SEARCH



ERROR: Error: ERROR: No se encontró el archivo gridsearch_grupodiez.csv en ninguna ubicación conocida
